In [1]:
from datetime import datetime
import json
import pandas as pd
import yaml

In [2]:
#gateway = "https://ipfs.filebase.io/ipfs/bafybeicxog6mwiga37znhopbxkwfbj5du4sp6rsga6vaht6y7cxugibpv4/"
gateway = "data/parquet/"

projects = pd.read_parquet(gateway+"projects.parquet")
projects.set_index('project_id', inplace=True)

votes = pd.read_parquet(gateway+"round_votes.parquet")
votes = (
    votes
    .groupby(['round_id', 'project_id'])
    .agg({'amount_usd': 'sum', 'grant_address': 'min', 'grant_address': lambda x: x.unique()[0]})
    .reset_index()
)

rounds = pd.read_parquet(gateway+"rounds.parquet")
rounds.set_index('id', inplace=True)
rounds = rounds[rounds['amount_usd'] > 0 ]
rounds_data = rounds.to_dict(orient='index')
round_name_mapping = rounds['name'].to_dict()

In [3]:
matching = pd.read_csv("data/csv/matching-distributions.csv")
matching['RoundId'] = matching['RoundId'].str.lower()
round_ids = list(matching['RoundId'].unique())
matching = matching.groupby(['RoundId', 'ProjectId'])['MatchAmountUSD'].sum()

In [4]:
address_records = json.load(open("data/validated_addresses.json"))
addresses = {}
for slug, adata in address_records.items():
    if slug == 'gitcoin':
        continue
    for addr in adata.keys():
        addresses.update({addr:slug})
len(addresses)        

17023

In [5]:
github_records = (
    pd.read_csv("data/csv/github_orgs_to_oso_slugs.csv", index_col=0)
    .groupby('github_org')['project_slug']
    .agg(lambda x: ", ".join(set(x)))
).to_dict()
#github_records.index.value_counts()
github_records['ethereum']

'protocol-guild, execution-spec-tests-ethereum, ethereum-cat-herders, ethereum-org-website-ethereum, sourcify-ethereum, js-ethereum-cryptography-ethereum, pm-ethereum, consensus-specs-ethereum, execution-specs-ethereum, fe-ethereum, utp-ethereum, remix-project, eth-portal-ethereum, distributed-validator-specs-ethereum, execution-apis-ethereum, trin-ethereum, beaconrunner-ethereum, glados-ethereum, portal-network-specs-ethereum, kzg-ceremony-specs-ethereum, portal-hive-ethereum, rig-ethereum, solidity, research-ethereum, go-ethereum, hive-ethereum, c-kzg-4844-ethereum, py-evm-ethereum'

In [6]:
missing_rounds = []
allo = {}
for project_id, row in projects.iterrows():
    md = row.get('metadata')
    if md == 'null':
        continue
    metadata = eval(md)
    rounddata = (
        votes[votes['project_id']==project_id]
        .set_index('project_id')
        .to_dict(orient='records')
    )
    for r in rounddata:
        
        round_id = r['round_id']
        round_data = rounds_data.get(round_id,{})
        r.update({
            'round_name': round_data.get('name'),
            'chain_id': round_data.get('chain_id'),
            'program_address': round_data.get('program_address'),
            'app_end_time': round_data.get('applications_end_time')
        })
        try:
            r.update({'match_usd': matching[(r['round_id'], project_id)]})
        except:
            r.update({'match_usd': None})
            if r['round_id'] not in round_ids:
                missing_rounds.append(r['round_id'])
            else:
                pass
    project_github = metadata.get('projectGithub')
    record = {
        'project_name': metadata['title'],
        'project_github': project_github,
        'rounds': rounddata
    }
    if rounddata:
        allo.update({project_id: record})

In [7]:
with open("data/gitcoin-allo.json", "w") as f:
    json.dump(allo, f, indent=2)

In [8]:
csv_data = []
for pid, pdata in allo.items():
    for rdata in pdata.get('rounds'):
        r = {
            'project_id': pid,
            'project_name': pdata['project_name'].encode('utf-8', 'replace').decode(),
            'project_github': pdata['project_github'],
            **rdata
        }
        a = r['grant_address'].lower()
        r.update({'oso_address_slug': addresses.get(a)})
        csv_data.append(r)
        
df = pd.DataFrame(csv_data)
df['project_github'] = df['project_github'].str.lower()
df['oso_github_slug'] = df['project_github'].map(github_records)

df = df[df['round_name'].str.contains("test|Test") == False]
df['round_date'] = df['app_end_time'].apply(lambda x: datetime.fromtimestamp(x))

cols = [
    'project_id', 
    'project_name', 
    'project_github', 
    'oso_github_slug',
    'grant_address',
    'oso_address_slug',
    'round_id',
    'round_name', 
    'chain_id',
    'program_address',
    'round_date',
    'amount_usd',  
    'match_usd', 
]
df = df[cols].set_index('project_id', drop=True)
df.head()

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,round_date,amount_usd,match_usd
project_id,,,,,,,,,,,,
0xe0f08b3b36137e01398e52e0db42d175732fede1624f364ffd3109acba81bef0,Crypto Sapiens,None,NaN,0x01999e431752136372a3d485f527907a6b02a1d1,None,0x2871742b184633f8dc8546c6301cbc209945033e,Web3 Community and Education,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-17 08:00:00,197.889151,598.929745
0xe0f08b3b36137e01398e52e0db42d175732fede1624f364ffd3109acba81bef0,Crypto Sapiens,None,NaN,0x624e05d9a8deff331d2685ef3d789b25f9bec80f,None,0x98720dd1925d34a2453ebc1f91c9d48e7e89ec29,Web3 Community and Education,424,0xe13da583181b19dace7c021f57774659edc1f901,2023-11-29 18:59:00,153.544471,611.130908
0x646d41be0fbbe228c01a54aecfde54250c0ae01d3de2221ce80971e8ff542a50,NFT Price Floor,nft-pricefloor,nft-pricefloor,0x31856e11ddaabc67e3b7b5de50dd0efce86e361e,None,0x98720dd1925d34a2453ebc1f91c9d48e7e89ec29,Web3 Community and Education,424,0xe13da583181b19dace7c021f57774659edc1f901,2023-11-29 18:59:00,307.369775,424.773680
0x8427d56c001d898a2afc097dfffbe471b26dfcc4bf8a9fac386266a5ed1ef37b,Ethereum News Podcast,ethdailyhub,NaN,0xeb40a065854bd90126a4e697aea0976ba51b2ee7,None,0x2871742b184633f8dc8546c6301cbc209945033e,Web3 Community and Education,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-17 08:00:00,248.087660,668.553916
0x8427d56c001d898a2afc097dfffbe471b26dfcc4bf8a9fac386266a5ed1ef37b,Ethereum News Podcast,ethdailyhub,NaN,0xeb40a065854bd90126a4e697aea0976ba51b2ee7,None,0x59d79b22595b17af659ce9b03907615f53742c57,"The Education, community growth & events Round",42161,0xce06ebb79f95943ca9125b85ca0294c302e846b5,2023-09-30 19:59:00,195.740634,1269.179638


In [9]:


df

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,round_date,amount_usd,match_usd
project_id,,,,,,,,,,,,
0xe0f08b3b36137e01398e52e0db42d175732fede1624f364ffd3109acba81bef0,Crypto Sapiens,None,NaN,0x01999e431752136372a3d485f527907a6b02a1d1,None,0x2871742b184633f8dc8546c6301cbc209945033e,Web3 Community and Education,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-17 08:00:00,197.889151,598.929745
0xe0f08b3b36137e01398e52e0db42d175732fede1624f364ffd3109acba81bef0,Crypto Sapiens,None,NaN,0x624e05d9a8deff331d2685ef3d789b25f9bec80f,None,0x98720dd1925d34a2453ebc1f91c9d48e7e89ec29,Web3 Community and Education,424,0xe13da583181b19dace7c021f57774659edc1f901,2023-11-29 18:59:00,153.544471,611.130908
0x646d41be0fbbe228c01a54aecfde54250c0ae01d3de2221ce80971e8ff542a50,NFT Price Floor,nft-pricefloor,nft-pricefloor,0x31856e11ddaabc67e3b7b5de50dd0efce86e361e,None,0x98720dd1925d34a2453ebc1f91c9d48e7e89ec29,Web3 Community and Education,424,0xe13da583181b19dace7c021f57774659edc1f901,2023-11-29 18:59:00,307.369775,424.773680
0x8427d56c001d898a2afc097dfffbe471b26dfcc4bf8a9fac386266a5ed1ef37b,Ethereum News Podcast,ethdailyhub,NaN,0xeb40a065854bd90126a4e697aea0976ba51b2ee7,None,0x2871742b184633f8dc8546c6301cbc209945033e,Web3 Community and Education,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-17 08:00:00,248.087660,668.553916
0x8427d56c001d898a2afc097dfffbe471b26dfcc4bf8a9fac386266a5ed1ef37b,Ethereum News Podcast,ethdailyhub,NaN,0xeb40a065854bd90126a4e697aea0976ba51b2ee7,None,0x59d79b22595b17af659ce9b03907615f53742c57,"The Education, community growth & events Round",42161,0xce06ebb79f95943ca9125b85ca0294c302e846b5,2023-09-30 19:59:00,195.740634,1269.179638
...,...,...,...,...,...,...,...,...,...,...,...,...
0x38c49fc81a116e993a49ffcdfed48bb1b75d4fdcb7994244f44adca553efabb1,Customized DeFi Dashboards,None,NaN,0xabf28f8d9adfb2255f4a059e37d3bce9104969db,None,0xe00cde3ce8e10049143ecea103bd8cd6641d67ca,We do a little democratic voting,250,0x27cdc959d083fa75171f51d1aab7d29fc3ebd7ea,2023-10-30 20:00:00,14.989519,NaN
0x2b3ab7db23e408acd50dc3777af52018461618c900225bd8a8e81a31e88d09a9,Dspyt-CodeVerse,dspytdao,dspytdao,0x4c11ba2ed1d936d769d0cce34cbc7ea1e85182d0,None,0x98720dd1925d34a2453ebc1f91c9d48e7e89ec29,Web3 Community and Education,424,0xe13da583181b19dace7c021f57774659edc1f901,2023-11-29 18:59:00,264.390610,85.418729
0xab931acdedab781ffea877def1aec0c8a4c0811dc24db51231178f3d497e91ac,ONE Human Coordination System,onelocalorg,NaN,0x6bd2dd6ba6ddc19687226e9dc30af311bcb44709,None,0x3558a83d59fdc4010c85c8cee6388a1b655f7336,Seed Round,250,0xb907ea87fc122397f8d9ef107919b5696ccf5192,2024-01-06 19:00:00,178.422459,NaN


In [10]:
df.to_csv("data/csv/gitcoin-allo.csv")

In [27]:
cgrants = pd.read_csv("data/csv/cGrantsByRound.csv")

,round_num,title,total_unique_contributors,total_contributions,total_crowdfund_amount,total_match_amount,total_funding,themes,payoutaddress,github,org_name,address
0,1,Burner Wallet,7.0,7.0,556.00,984.69,1540.69,['gitcoin-main'],0x78040Ab06E05F59FD78Ca7367cbf4E95069cAc47,NaN,NaN,NaN
1,1,ChainID Network,1.0,1.0,0.10,0.06,0.16,['gitcoin-main'],0xFcD819c8A13de2d9E409FDf7D61161091F58dD63,NaN,NaN,NaN
2,1,Connext Network,1.0,1.0,1570.76,738.26,2309.02,['gitcoin-main'],0xc37E165fCFA2cb5cCa7329dD9DAa8Cd72907b983,NaN,NaN,NaN
3,1,"Cryptoeconomics.study - Free, Open-Source Bloc...",15.0,15.0,361.00,2071.84,2432.84,['gitcoin-main'],0x3efcF2dDD54D841F253a95d8f30d97123568Cd7B,NaN,NaN,NaN
4,1,ETH2.0 Implementers Call Notes,5.0,5.0,121.20,202.83,324.03,['gitcoin-main'],0xFd3FfE624268a88F5FAe682BfED8190b74d8C6F6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15833,16,word block,82.0,86.0,211.51,470.73,682.24,['oss'],NaN,https://github.com/lg565245594,lg565245594,0x65a0af703047dfdd270361659d02f4f0e8547202
15834,16,WTF Academy,382.0,393.0,1193.89,1420.16,2614.05,['oss'],NaN,https://github.com/WTFAcademy,WTFAcademy,0x25df6da2f4e5c178ddff45038378c0b08e0bce54
15835,16,Zero Knowledge Podcast,677.0,688.0,1448.55,1551.77,3000.32,['oss'],NaN,https://github.com/ZeroKnowledgefm,ZeroKnowledgefm,0x0b0e6486648fbbb8e9ab33e3ae9d0b44b6faa701
15836,16,ZeroPool,1315.0,1338.0,2982.00,4324.04,7306.04,['oss'],NaN,https://github.com/zeropoolnetwork,zeropoolnetwork,0x947f8a49640b4770a8c7fe3c1e69ffc974295448


# Join on cGrants

In [30]:
cgrants = pd.read_csv("data/csv/cGrantsByRound.csv")
dgrants = pd.read_csv("data/csv/AlphaRoundFundingWithWalletsGithubs.csv")
cgrants = pd.concat([cgrants, dgrants], axis=0, ignore_index=True)

cgrants['payoutaddress'] = cgrants['payoutaddress'].str.lower()

cgrant_timings = [
    {"_mb_row_id":1,"Round Num":1,"Start Date":"2019-02-01T00:00:00Z","End Date":"2019-02-15T00:00:00Z"},
    {"_mb_row_id":2,"Round Num":2,"Start Date":"2019-03-26T00:00:00Z","End Date":"2019-04-19T00:00:00Z"},
    {"_mb_row_id":3,"Round Num":3,"Start Date":"2019-09-15T00:00:00Z","End Date":"2019-10-04T00:00:00Z"},
    {"_mb_row_id":4,"Round Num":4,"Start Date":"2020-01-06T00:00:00Z","End Date":"2020-01-21T00:00:00Z"},
    {"_mb_row_id":5,"Round Num":5,"Start Date":"2020-03-23T00:00:00Z","End Date":"2020-04-05T00:00:00Z"},
    {"_mb_row_id":6,"Round Num":6,"Start Date":"2020-06-16T00:00:00Z","End Date":"2020-07-03T00:00:00Z"},
    {"_mb_row_id":7,"Round Num":7,"Start Date":"2020-09-14T00:00:00Z","End Date":"2020-10-02T18:00:00Z"},
    {"_mb_row_id":8,"Round Num":8,"Start Date":"2020-12-02T00:00:00Z","End Date":"2020-12-18T00:00:00Z"},
    {"_mb_row_id":9,"Round Num":9,"Start Date":"2021-03-10T00:00:00Z","End Date":"2021-03-26T00:00:00Z"},
    {"_mb_row_id":10,"Round Num":10,"Start Date":"2021-06-16T15:00:00Z","End Date":"2021-07-02T00:00:00Z"},
    {"_mb_row_id":11,"Round Num":11,"Start Date":"2021-09-08T15:00:00Z","End Date":"2021-09-24T00:00:00Z"},
    {"_mb_row_id":12,"Round Num":12,"Start Date":"2021-12-01T15:00:00Z","End Date":"2021-12-17T00:00:00Z"},
    {"_mb_row_id":13,"Round Num":13,"Start Date":"2022-03-09T15:00:00Z","End Date":"2022-03-25T00:00:00Z"},
    {"_mb_row_id":14,"Round Num":14,"Start Date":"2022-06-08T15:00:00Z","End Date":"2022-06-24T00:00:00Z"},
    {"_mb_row_id":15,"Round Num":15,"Start Date":"2022-09-07T15:00:00Z","End Date":"2022-09-23T00:00:00Z"},
    {"_mb_row_id":16,"Round Num":16,"Start Date":"2023-01-17T15:00:00Z","End Date":"2023-01-31T00:00:00Z"}
]
timings = {
    x["Round Num"]: x["Start Date"]
    for x in cgrant_timings
}

def get_github(x):
    if isinstance(x, str):
        x = x.lower()
        if 'github.com/' in x:
            x = x.split('github.com/')[1]
            x = x.split("/")[0]
            if x != 'orgs':
                return x
cgrants['project_github'] = cgrants['github'].apply(get_github)
cgrants['oso_github_slug'] = cgrants['project_github'].map(github_records)
cgrants['oso_address_slug'] = cgrants['payoutaddress'].map(addresses)

cgrants.rename(columns={
    'title': 'project_name', 
    'payoutaddress': 'grant_address', 
    'total_crowdfund_amount': 'amount_usd',
    'total_match_amount': 'match_usd',
    'round_num': 'round_id',
    'themes': 'round_name'
}, inplace=True)

cgrants = cgrants[[
    'project_name', 'project_github', 'oso_github_slug', 'grant_address',
    'oso_address_slug', 'round_id', 'round_name', 'amount_usd', 'match_usd'
]]

cgrants['round_date'] = cgrants['round_id'].map(timings)
cgrants.head(2)

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,amount_usd,match_usd,round_date
0,Burner Wallet,None,NaN,0x78040ab06e05f59fd78ca7367cbf4e95069cac47,NaN,1,['gitcoin-main'],556.0,984.69,2019-02-01T00:00:00Z
1,ChainID Network,None,NaN,0xfcd819c8a13de2d9e409fdf7d61161091f58dd63,NaN,1,['gitcoin-main'],0.1,0.06,2019-02-01T00:00:00Z


In [31]:
all_grants = pd.concat([df, cgrants], axis=0, ignore_index=True)
all_grants.head(1)

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,round_date,amount_usd,match_usd
0,Crypto Sapiens,None,NaN,0x01999e431752136372a3d485f527907a6b02a1d1,None,0x2871742b184633f8dc8546c6301cbc209945033e,Web3 Community and Education,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-17 08:00:00,197.889151,598.929745


In [32]:
all_grants['total_usd'] = all_grants['amount_usd'] + all_grants['match_usd']

In [33]:
all_grants['total_usd'].sum()

38184013.34814828

In [34]:
def match_slug(github_slug, address_slug):
    if isinstance(address_slug, str):
        if isinstance(github_slug, str):
            if address_slug in github_slug:
                return address_slug
            else:
                return ", ".join([address_slug, github_slug])
        else:
            return address_slug
    else:
        return github_slug
    
all_grants['oso_slug'] = all_grants.apply(lambda x: match_slug(x['oso_github_slug'], x['oso_address_slug']), axis=1)
all_grants.head(1)

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,round_date,amount_usd,match_usd,total_usd,oso_slug
0,Crypto Sapiens,None,NaN,0x01999e431752136372a3d485f527907a6b02a1d1,None,0x2871742b184633f8dc8546c6301cbc209945033e,Web3 Community and Education,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-17 08:00:00,197.889151,598.929745,796.818896,NaN


In [36]:
oso_grants = all_grants[all_grants['oso_slug'].isna()==False]
top100 = oso_grants.groupby('oso_slug')['total_usd'].sum().sort_values().tail(100)
toprnd = oso_grants.groupby('oso_slug')['round_id'].nunique()
toprnd = toprnd[toprnd > 4]

In [38]:
top100_slugs = set([i for lst in top100.index for i in lst.split(", ")])
toprnd_slugs = set([i for lst in toprnd.index for i in lst.split(", ")])

top_slugs = top100_slugs.intersection(toprnd_slugs)
top_slugs.remove('protocol-guild')
top_slugs

{'4everland',
 'astralprotocol',
 'banklessdao',
 'beacon-chain',
 'blockhead-darryl-yeo',
 'blockscout',
 'bloomnetwork',
 'brightid',
 'buidlguidl',
 'chainlist-defillama',
 'clrfund',
 'commons-stack',
 'crypto-fees-dmihal',
 'dapp-learning-dao',
 'dappnode',
 'dataverse-os',
 'defi-llama',
 'defieye',
 'dna-seq',
 'edendao',
 'electronic-frontier-foundation',
 'esteroids',
 'eth-limo',
 'eth-wizard-stake-house',
 'ethers-io',
 'ethhub-io',
 'ethstaker',
 'fileverse',
 'frame-floating',
 'freedom-of-the-press-foundation',
 'gaia-os',
 'gainforest',
 'geo-web',
 'gitcoin',
 'giveth',
 'hardhat-nomicfoundation',
 'idena-network',
 'idriss-crypto',
 'jedi-swap',
 'krebitdao',
 'kredeum',
 'l2beat',
 'leastauthority',
 'lenstube-xyz',
 'lexdao',
 'lifinance',
 'lighthouse-sigp',
 'loanshark',
 'metamail',
 'minerva-wallet-lab10-coop',
 'nethermindeth',
 'nft-pricefloor',
 'nicenode',
 'optinames-optimismresolver',
 'pepemon-dao',
 'poapin-glory-lab',
 'proof-of-humanity',
 'prysmaticlab

In [39]:
sample = oso_grants[oso_grants['oso_slug'].isin(top_slugs)]
sample.to_csv("data/csv/gitcoin_funding_events.csv")

# Backup

In [19]:
oso_grants[oso_grants['oso_slug'].str.contains(', ') == True]['oso_slug'].value_counts()

oso_slug
filecoin-project, starling-filecoin-project                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      45
commonsbuild, gravity-dao-archive                                                                                                                                                                                                                                                                                                                                                                          

In [ ]:
oso_grants['oso_slug']

In [101]:
slugs = []
for s in oso_grants['oso_slug'].unique():
    if ", " in s:
        ss = s.split(", ")
        slugs.extend(ss)
    else:
        slugs.append(s)
        
set(slugs)        

{'0u0-ai',
 '0x-boring-security',
 '0x-dead-list',
 '0x-echo',
 '0x3327',
 '0xbootleggers',
 '0xdeadlist',
 '0xdeschool',
 '0xeigenlabs',
 '0xfableorg',
 '0xfundstarter',
 '0xnextlabs',
 '0xparc',
 '0xrivendell',
 '153x153x153',
 '1hive',
 '1w3',
 '404-dao',
 '4everland',
 '4seascm',
 '88-mph',
 'accountjs',
 'aikipay',
 'akula-bft',
 'alasdao',
 'aletheia-data',
 'algoveraai',
 'all-weather-protocol',
 'alto-io',
 'ambiretech',
 'amphitheatre-app',
 'ankr-network',
 'antalpha-labs',
 'aoraki-labs',
 'ape-framework',
 'aqkuperu',
 'archlyorg',
 'arm0ry',
 'asilodigital',
 'asseth',
 'astralprotocol',
 'atilatech',
 'atomxyz',
 'attentionadsqap',
 'atticc',
 'atxdao',
 'auditdbio',
 'aura-meta-node',
 'autochessia-helheimlabs',
 'aw-labs',
 'backpack-app',
 'bankless-academy',
 'bankless-africa',
 'banklessdao',
 'banyancomputer',
 'basexhq',
 'basin-global',
 'beacon-chain',
 'beaconrunner-ethereum',
 'beepo-app',
 'beepopula',
 'beethoven-x',
 'benzin-web3-oss',
 'better-feedback',
 '

In [71]:
oso_grants[oso_grants['oso_github_slug'] == 'rotki']

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,app_end_time,amount_usd,match_usd,total_usd,slug
488,rotki,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,0x64e5b2228ef31437909900b38fc42dd5e4b83147,ENS Ecosystem,1,0xdb723e5fe784c160a89564d22d67a11eb08cf76d,1.682122e+09,177.362279,2503.364104,2680.726383,None
489,rotki,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,0xdf22a2c8f6ba9376ff17ee13e6154b784ee92094,Ethereum Infrastructure,1,0xa1b6245d7ba4b126adf7ee1e05e96bfda974990c,1.682122e+09,2195.841326,19980.639587,22176.480913,None
2044,rotki,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,0x8de918f0163b2021839a8d84954dd7e8e151326d,Web3 Open Source Software,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,1.692274e+09,3777.971893,1227.545685,5005.517578,None
2045,rotki,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,0xa1d52f9b5339792651861329a046dd912761e9a9,"Ethereum Core Infrastructure, Research, and De...",137,0xd04d5b0257a9f2068a4f4ebd88748b2bb47da4ca,1.701302e+09,3392.750122,10359.880968,13752.631090,None
3334,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,4,['gitcoin-main'],NaN,NaN,NaN,934.960000,3027.000000,3961.960000,None
3636,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,5,['gitcoin-main'],NaN,NaN,NaN,580.380000,3165.090000,3745.470000,None
4181,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,6,['gitcoin-main'],NaN,NaN,NaN,2019.960000,2507.680000,4527.640000,None
4646,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,7,['gitcoin-main'],NaN,NaN,NaN,6828.810000,21721.230000,28550.040000,None
5184,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,8,['gitcoin-main'],NaN,NaN,NaN,4454.460000,7792.520000,12246.980000,None
5988,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,9,['gitcoin-main'],NaN,NaN,NaN,14119.000000,5375.800000,19494.800000,None


In [60]:
def assign_slug(git, addr):
    gits = set()
    addrs = set()
    if isinstance(addr, str):
        addrs = set(addr)
    if isinstance(git, str):
        if ", " in git:
            #print(git)
            gits = set(list(git.split(", ")))
        else:
            print()
            gits = set(git)
    
    if gits and addrs:
        result = gits.intersection(addrs)
    elif addrs:
        result = addrs
    else:
        result = gits
    
    if len(result) > 1:
        return None
    elif result:
        return result
    else:
        return None

all_grants['slug'] = all_grants.apply(
    lambda x: assign_slug(x['oso_github_slug'], x['oso_address_slug']), axis=1
)
            

gravity-dao-archive, commonsbuild
pairwise-general-magic, generalmagicio
pairwise-general-magic, generalmagicio
revoke-cash, revokecash
zenbiteth, zenbit-eth, eneagon-zenbit-eth
ens-wayback-machine, blossomlabs
elefria-labs, zk-block-elefria-labs
elefria-labs, zk-block-elefria-labs
elefria-labs, zk-block-elefria-labs
lumeweb, lume-web
lumeweb, lume-web
lumeweb, lume-web
poapin-glory-lab, glorylab
poapin-glory-lab, glorylab
poapin-glory-lab, glorylab
poapin-glory-lab, glorylab
poapin-glory-lab, glorylab
poapin-glory-lab, glorylab
poapin-glory-lab, glorylab
poapin-glory-lab, glorylab
poapin-glory-lab, glorylab
poapin-glory-lab, glorylab
0xdeadlist, 0x-dead-list
0xdeadlist, 0x-dead-list
starksheet, the-candy-shop
starksheet, the-candy-shop
starksheet, the-candy-shop
fat-garage, word-block-fat-garage
ape-framework, apeworx
rouge-ticket, therougeproject
revoke-cash, revokecash
defi-llama, chainlist-defillama
defi-llama, chainlist-defillama
eoa-blockchain-labs, ethereum-on-arm
eoa-blockchain

In [49]:
set("5").pop()

'5'

# Join on other grants data

In [11]:
op = pd.read_csv("../optimism/grants/2024-02-06_op_tracked_grants.csv", index_col=0)
op.head(1)

,project,link,amount,slug,tags,dune_namespace
0,0xSplits,https://app.optimism.io/retropgf-discovery/0x9...,52154.0,0x-splits,RetroPGF2,0xSplits


In [13]:
evm = pd.read_csv("data/csv/evm_grants.csv")
evm.iloc[0]

grant_source                                  Arbitrum Foundation
grantee                                             Gains Network
grant_date                                               1/1/2024
grant_token_address    0x912ce59144191c1204e64559fe8253a0e49e6548
grant_amount                                              4500000
grant_name                                                   STIP
grant_round                                                     1
grant_type                                              proactive
grant_category                                             growth
grant_distribution                                          claim
grant_blockchain                                         arbitrum
dune_namespaces            gains,gains_network,gains_network_usdc
dashboard_link                                                NaN
information_link                                              NaN
Name: 0, dtype: object